In [ ]:
# RESTART KERNEL AFTER INSTALLING
!pip install --quiet openai

In [ ]:
from IPython.display import Markdown
import pandas as pd
from getpass import getpass
from openai import OpenAI

In [ ]:
openai_key = getpass("Wprowadź swój klucz OpenAI: ")

In [ ]:
openai_client = OpenAI(api_key=openai_key)

def ASK(prompt: str):
    ASK_SENIOR_DATA_SCIENTIST_PROMPT = """
        * Jesteś Senior Data Scientistem w dużym przedsiębiorstwie.
        * Twoim zadaniem jest pomoc i mentorowanie młodszych pracowników, takich jak ja
        * Będę cię prosił o różne rady i wskazówki, które pomogą mi w mojej pracy
        * Odpowiadaj zwięźle i jeżeli się da od razu przesyłaj mi kod w Pythonie
    """
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        messages=[
            {"role": "system", "content": ASK_SENIOR_DATA_SCIENTIST_PROMPT},
            {"role": "user", "content": prompt}
        ]
    )

    return Markdown(response.choices[0].message.content)

In [ ]:
# sprawdźmy czy nasz mentor działa
ASK("Jak posortować rzędy w DataFrame po wartościach w kolumnie 'price'?")

# Naprawiamy dane z ankiety `Poznajmy się lepiej`

* W lekcji tej zajmiemy się danymi z ankiety `Poznajmy się lepiej`. 
* Ankieta była skonstruowana tak, aby dopuszczać wiele odpowiedzi wpisywanych bezpośrednio przez uczestników.
* Prezentujemy tylko fragment danych, aby nie naruszać prywatności uczestników.
* Dodatkowo dane zostały nieco zmodyfikowane, aby nie było możliwe identyfikowanie uczestników.

In [ ]:
df = pd.read_csv('welcome_survey.csv')

df.head()

In [ ]:
df = df.rename(columns={
    "Poziom wykształcenia": "edu_level",
    "Obecna branża": "industry",
    'Liczba lat doświadczenia zawodowego': 'years_of_experience',
    'Jak preferujesz się uczyć? (wybierz wszystkie, które pasują)': 'learning_preferences',
    'Ile czasu tygodniowo możesz poświęcić na naukę data science i AI?': 'weekly_study_hours',
    'Co najbardziej motywuje Cię do nauki data science i AI? (wybierz maks. 3)': 'motivation',
    'Jak zwierzęta są Twoimi ulubionymi?': 'fav_animals',
    'Jakie jest Twoje ulubione miejsce na ziemi?' : 'fav_place',
    'Bardziej lubisz przysmaki słodkie czy słone?': 'sweet_or_salty',
    'Płeć': 'gender',
    'Wiek': 'age',
    'Jak spędzasz wolny czas?' : 'hobbies',
})
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

## Naprawiamy / czyścimy kolumnę `Wiek`

In [ ]:
df["age"].value_counts()

In [ ]:
df["age_cleaned"] = df["age"].map({
    "Poniżej 18 lat": "<18",
    "18-24": "18-24",
    "25-34": "25-34",
    "35-44": "35-44",
    "45-54": "45-54",
    "55-64": "55-64",
    "65 i więcej": ">=65",
    "Nie chcę podawać": "unknown"
})
df["age_cleaned"] = df["age_cleaned"].fillna("unknown")
df["age_cleaned"].value_counts()

## Naprawiamy / czyścimy kolumnę `Płeć`

In [ ]:
df["gender"].value_counts()

In [ ]:
df["gender_cleaned"] = df["gender"].map({
    "Mężczyzna": 0,
    "Kobieta": 1,
})
df["gender_cleaned"].value_counts()

## Naprawiamy / czyścimy kolumnę `Liczba lat doświadczenia zawodowego`

In [ ]:
df["years_of_experience"].value_counts()

In [ ]:
df["years_of_experience_cleaned"] = df["years_of_experience"].map({
    "0-2 lata": "0-2",
    "3-5 lat": "3-5",
    "6-10 lat": "6-10",
    "11-15 lat": "11-15",
    "16 i więcej": ">=16",
})
df["years_of_experience_cleaned"].value_counts()

## Naprawiamy / czyścimy kolumnę `Jak spędzasz wolny czas?`

In [ ]:
df["hobbies"].value_counts()

In [ ]:
df["hobbies"].str.get_dummies(sep=", ").sum().sort_values(ascending=False)

In [ ]:
def categorize_hobbies(hobbies):
    if not isinstance(hobbies, str):
        return []

    cleaned = []
    for hobby in hobbies.split(", "):
        if hobby == "Filmy / Seriale":
            cleaned.append("hobby_movies")
        elif hobby == "Sport":
            cleaned.append("hobby_sport")
        elif hobby == "Książki":
            cleaned.append("hobby_books")
        elif hobby == "Gry komputerowe":
            cleaned.append("hobby_video_games")
        elif hobby == "Sztuka":
            cleaned.append("hobby_art")
        else:
            cleaned.append("hobby_other")

    return ", ".join(cleaned)


df["hobbies_cleaned"] = df["hobbies"].apply(categorize_hobbies)
df["hobbies_cleaned"].str.get_dummies(sep=", ").sum().sort_values(ascending=False)

In [ ]:
hobbies_df = df["hobbies_cleaned"].str.get_dummies(sep=", ")
hobbies_df = hobbies_df.drop(columns=["[]"])
hobbies_df

In [ ]:
df = pd.concat([df, hobbies_df], axis=1)

## Naprawiamy / czyścimy kolumnę `Bardziej lubisz przysmaki słodkie czy słone?`

In [ ]:
df["sweet_or_salty"].value_counts()

In [ ]:
df["sweet_or_salty_cleaned"] = df["sweet_or_salty"].map({
    "Słone": "salty",
    "Słodkie": "sweet",
})
df["sweet_or_salty_cleaned"].value_counts()

## Naprawiamy / czyścimy kolumnę `Jakie jest Twoje ulubione miejsce na ziemi?`

In [ ]:
df["fav_place"].value_counts()

In [ ]:
", ".join(df["fav_place"].dropna().unique().tolist())

In [ ]:
ASK("""
W kolumnie 'Jakie jest Twoje ulubione miejsce na ziemi?' mam różne wartości. Niektóre można by zgrupować w jedną kategorię,
np. 'Nad wodą', 'W górach' .etc
Oto wartości w tej kolumnie:
    Morze, Las, Góry, las i jezioro, Zielona oaza siedliskowa nad wodą., Miasto, ruch, spacer nad morzem, dom,
    we wszystkich miejscach czuje sie dobrze, Jezioro, Dom, Góry połączone z jeziorem, wieś, Morze Las , góry, morze, las, Teren nad rzeką ,
    Albania , Mazury, Góry, Morze, Las, miejsca, gdzie jest mało ludzi, teren gdzie temperatura nie spada do 20 stopni,
    góry + zbiornik wodny (np. Jezioro Garda), Każde z powyższych, Wyspy / Ocean, Las i jezioro

Jak zgrupować te wartości do mniejszej liczby kategorii?
""")

In [ ]:
category_map = {
    'Morze': 'Nad wodą',
    'Jezioro': 'Nad wodą',
    'Teren nad rzeką': 'Nad wodą',
    'Wyspy / Ocean': 'Nad wodą',
    'las i jezioro': 'Nad wodą',
    'góry + zbiornik wodny (np. Jezioro Garda)': 'Nad wodą',
    'Góry': 'W górach',
    'Góry połączone z jeziorem': 'W górach',
    'Las': 'W lesie',
    'Las i jezioro': 'W lesie',
    'Morze Las': 'W lesie',
    'Miasto, ruch': 'Miasto',
    'dom': 'Inne',
    'wieś': 'Inne',
    'miejsca, gdzie jest mało ludzi': 'Inne',
    'teren gdzie temperatura nie spada do 20 stopni': 'Inne',
    'Każde z powyższych': 'Inne',
    'Albania': 'Inne',
    'Mazury': 'Inne',
    'spacer nad morzem': 'Inne',
    'we wszystkich miejscach czuje sie dobrze': 'Inne',
    'Dom': 'Inne',
}

df['fav_place_cleaned'] = df['fav_place'].map(category_map)
df['fav_place_cleaned'].value_counts()

## Naprawiamy `Poziom wykształcenia`

In [ ]:
df["edu_level"].value_counts()

In [ ]:
len(df["edu_level"].dropna().unique())

In [ ]:
", ".join(df["edu_level"].dropna().unique().tolist())

In [ ]:
ASK("""
W kolumnie 'Poziom wykształcenia' mam różne wartości. Niektóre można by zgrupować w jedną kategorię,
np. 'Podstawowe', 'Średnie', 'Wyższe'.
Oto wartości w tej kolumnie:
    Szkoła podstawowa, Szkoła średnia / Technikum, Magister, Inżynier, Doktorat, mgr inż., Licencjat,
    Studia podyplomowe, Szkoła zawodowa, inżynier, Inżynier , magister inżynier, W szkole doktorskiej, Inżynierskie + podyplomowe

Jak zgrupować te wartości w mniej kategorii?
""")

In [ ]:
def group_education(level):
    if level in ['Szkoła podstawowa']:
        return 'Podstawowe'
    elif level in ['Szkoła średnia / Technikum', 'Szkoła zawodowa']:
        return 'Średnie'
    else:
        return 'Wyższe'

# Zastosowanie funkcji do kolumny
df['edu_level_cleaned'] = df['edu_level'].apply(group_education)

In [ ]:
df['edu_level_cleaned'].value_counts()

## Naprawiamy `Obecna branża`

In [ ]:
df["industry"].value_counts()

In [ ]:
len(df["industry"].dropna().unique())

In [ ]:
", ".join(df["industry"].dropna().unique().tolist())

In [ ]:
ASK("""
Dla każdej z podanych nazw przypisz branżę:
    IT, Edukacja, Energetyka, Automotvie, Automatyzacja, Energetyka zawodowa, Zdrowie, Kadry (HR), Marketing, Produkcja, Wellness,
    Chemia, Nieruchomości, poligrafia, Administracja publiczna, chemik branża automotive, usługi, obsługa klienta, brak,
    Budowlana, Automatyka i robotyka, Bezrobotny, Finanse, izynier GPS, Opieka, emeryt, Hotelarstwo, Budowa maszyn , energetyka,
    Logistyka, Branża Produkcyjno Usługowa, Motoryzacja, Media, fotografia, wideo dron, Bez pracy, ostatnio HR, logistyka,
    zakupy w dystrybucji IT, R&D, przemysł - zarządzanie jakością, Inżyniera, Pracownik naukowy: Nawigacja samolotowa,
    systemy nawigacji satelitarnej, administrator sieci i systemów w służbie zdrowia, FMCG , produkcja, fundusze europejskie,
    Administracja , Projektowanie mebli na wymiar, Logistyka , Ochrona Środowiska, Automotiv,
    Pomoc Społeczna (Asystent osobisty osoby z niepełnosprawnością), Zabezpieczenia transportowe, Logistyka i Produkcja, Transport,
    Property Management, księgowość, Bezpieczeństwo, Produkcja , Mechanik, Logistyka i Transport, handel, budownictwo/architektura,
    Usługi, Transport Międzynarodowy, Chłodnictwo, Marketing, SEO, logistyka, usługi elektryczne, Architektura/Urbansityka,
    Inżynieria, e-commerce, Budownictwo, fotowoltaika, Mechanik Samochodowy, Gastronomia,
    Pracuję przy projektach związanych z sieciami gazowymi, Spedycja

Odpowiedź zwróć jako słownik pythona, gdzie kluczem jest podana przeze mnie nazwa, a wartością przypisana branża.
""")

In [ ]:
df["industry_cleaned"] = df["industry"].map({
    "IT": "IT",
    "Edukacja": "Edukacja",
    "Energetyka": "Energetyka",
    "Automotvie": "Automotive",
    "Automatyzacja": "Automatyzacja",
    "Energetyka zawodowa": "Energetyka zawodowa",
    "Zdrowie": "Zdrowie",
    "Kadry (HR)": "Kadry (HR)",
    "Marketing": "Marketing",
    "Produkcja": "Produkcja",
    "Wellness": "Wellness",
    "Chemia": "Chemia",
    "Nieruchomości": "Nieruchomości",
    "poligrafia": "Poligrafia",
    "Administracja publiczna": "Administracja publiczna",
    "chemik branża automotive": "Automotive",
    "usługi": "Usługi",
    "obsługa klienta": "Obsługa klienta",
    "brak": "Brak",
    "Budowlana": "Budowlana",
    "Automatyka i robotyka": "Automatyka i robotyka",
    "Bezrobotny": "Bezrobotny",
    "Finanse": "Finanse",
    "izynier GPS": "Inżynieria",
    "Opieka": "Opieka",
    "emeryt": "Emerytura",
    "Hotelarstwo": "Hotelarstwo",
    "Budowa maszyn": "Budowa maszyn",
    "energetyka": "Energetyka",
    "Logistyka": "Logistyka",
    "Branża Produkcyjno Usługowa": "Branża Produkcyjno Usługowa",
    "Motoryzacja": "Motoryzacja",
    "Media": "Media",
    "fotografia": "Fotografia",
    "wideo dron": "Wideo dron",
    "Bez pracy": "Bez pracy",
    "ostatnio HR": "Kadry (HR)",
    "logistyka": "Logistyka",
    "zakupy w dystrybucji IT": "IT",
    "R&D": "R&D",
    "przemysł - zarządzanie jakością": "Produkcja",
    "Inżyniera": "Inżynieria",
    "Pracownik naukowy: Nawigacja samolotowa": "Nauka",
    "systemy nawigacji satelitarnej": "Inżynieria",
    "administrator sieci i systemów w służbie zdrowia": "Zdrowie",
    "FMCG": "FMCG",
    "produkcja": "Produkcja",
    "fundusze europejskie": "Finanse",
    "Administracja": "Administracja",
    "Projektowanie mebli na wymiar": "Usługi",
    "Logistyka": "Logistyka",
    "Ochrona Środowiska": "Ochrona Środowiska",
    "Automotiv": "Automotive",
    "Pomoc Społeczna (Asystent osobisty osoby z niepełnosprawnością)": "Pomoc Społeczna",
    "Zabezpieczenia transportowe": "Transport",
    "Logistyka i Produkcja": "Logistyka i Produkcja",
    "Transport": "Transport",
    "Property Management": "Nieruchomości",
    "księgowość": "Finanse",
    "Bezpieczeństwo": "Bezpieczeństwo",
    "Produkcja": "Produkcja",
    "Mechanik": "Automotive",
    "Logistyka i Transport": "Logistyka i Transport",
    "handel": "Usługi",
    "budownictwo/architektura": "Budownictwo",
    "Usługi": "Usługi",
    "Transport Międzynarodowy": "Transport",
    "Chłodnictwo": "Usługi",
    "Marketing": "Marketing",
    "SEO": "Marketing",
    "logistyka": "Logistyka",
    "usługi elektryczne": "Usługi",
    "Architektura/Urbansityka": "Architektura",
    "Inżynieria": "Inżynieria",
    "e-commerce": "E-commerce",
    "Budownictwo": "Budownictwo",
    "fotowoltaika": "Energetyka",
    "Mechanik Samochodowy": "Automotive",
    "Gastronomia": "Gastronomia",
    "Pracuję przy projektach związanych z sieciami gazowymi": "Energetyka",
    "Spedycja": "Logistyka"
})

In [ ]:
len(df["industry_cleaned"].unique())

In [ ]:
df["industry_cleaned"].value_counts()

## Naprawiamy `Jak zwierzęta są Twoimi ulubionymi?`

In [ ]:
df["fav_animals"].value_counts()

In [ ]:
len(df["fav_animals"].dropna().unique())

In [ ]:
def categorize_animals(animal_name):
    if not isinstance(animal_name, str):
        return "Brak ulubionych"

    animal_name = animal_name.lower()
    if "Nie mam" in animal_name or "brak" in animal_name or "nie lubię" in animal_name:
        return "Brak ulubionych"

    elif "koty" in animal_name and "psy" in animal_name:
        return "Koty i Psy"

    elif "koty" in animal_name:
        return "Koty"

    elif "psy" in animal_name:
        return "Psy"

    else:
        return "Inne"

df["fav_animals_cleaned"] = df["fav_animals"].apply(categorize_animals)

In [ ]:
df["fav_animals_cleaned"].value_counts()

## Naprawiamy kolumnę `Jak preferujesz się uczyć? (wybierz wszystkie, które pasują)`

In [ ]:
df["learning_preferences"].value_counts()

In [ ]:
learning_pref_df = df["learning_preferences"].str.replace(", stacjonarnie", ", Kursy stacjonarne").str.get_dummies(sep=", ")
learning_pref_df = learning_pref_df.rename(columns={
    "Książki": "learning_pref_books",
    "Kursy online": "learning_pref_online_courses",
    "Kursy stacjonarne": "learning_pref_offline_courses",
    "Praca z ChatGPT": "learning_pref_chatgpt",
    "Praca zespołowa": "learning_pref_teamwork",
    "Samodzielne projekty": "learning_pref_personal_projects",
    "Uczenie innych osób." : "learning_pref_teaching",
    "Warsztaty": "learning_pref_workshops",
})
df = pd.concat([df, learning_pref_df], axis=1)
df.head()

## Naprawiamy kolumnę `Co najbardziej motywuje Cię do nauki data science i AI? (wybierz maks. 3)`

In [ ]:
df["motivation"].str.get_dummies(sep=", ").columns.tolist()

In [ ]:
values = df["motivation"].str.get_dummies(sep=", ").columns.tolist()
",".join(values)

In [ ]:
ASK("""
W kolumnie 'Co najbardziej motywuje Cię do nauki data science i AI' mam różne wartości. Niektóre można by zgrupować w jedną kategorię,
Oto wartości w tej kolumnie:
    Chęć zmiany zawodu,Dodanie kompetencji Data Science do obecnego profilu IT,Fascynacja możliwościami jakie daje nam AI,
    Pasja do analizy danych,Pomoc w realizacji swoich pomysłów,Rozwiązywanie rzeczywistych problemów,Rozwój kariery,Swoboda pracy,
    Wynagrodzenie,Wyzwania intelektualne,Zagadnienia w doktoracie,jako dźwigni zwiększającej mój projektowy zasięg (getting out of the box).,
    możliwośc tworzenia narzędzi AI,możliwość pracy zdalnej,odnalezieniesię na rynku pracy  ,pasja do AI,poznawanie nowych rzeczy,
    praca z AI,praca zdalna,rozwój osobisty,tworzenie aplikacji AI,własny projekt,zdobycie nowych umiejętności,
    znalezienie właściwej dla siebie drogi dla zmiany sytuacji zawodowej

Jak zgrupować te wartości w mniej kategorii? Stwórz słownik pythona, gdzie kluczem jest podana przeze mnie nazwa, a wartością przypisana motywacja.
""")

In [ ]:
motywacje = {
    "motivation_career": [
        "Chęć zmiany zawodu",
        "Znalezienie właściwej dla siebie drogi dla zmiany sytuacji zawodowej",
        "Dodanie kompetencji Data Science do obecnego profilu IT"
    ],
    "motivation_personal_growth": [
        "Fascynacja możliwościami jakie daje nam AI",
        "Pasja do analizy danych",
        "Rozwój kariery",
        "Rozwój osobisty",
        "Zdobycie nowych umiejętności",
        "Poznawanie nowych rzeczy"
    ],
    "motivation_remote": [
        "Swoboda pracy",
        "Możliwość pracy zdalnej",
        "Praca zdalna"
    ],
    "motivation_challenges": [
        "Rozwiązywanie rzeczywistych problemów",
        "Wyzwania intelektualne",
        "Zagadnienia w doktoracie, jako dźwigni zwiększającej mój projektowy zasięg (getting out of the box)"
    ],
    "motivation_creativity_and_innovation": [
        "Pomoc w realizacji swoich pomysłów",
        "Możliwość tworzenia narzędzi AI",
        "Tworzenie aplikacji AI",
        "Własny projekt"
    ],
    "motivation_money_and_job": [
        "Wynagrodzenie",
        "Odnalezienie się na rynku pracy"
    ]
}

def categorize_motivation(motivation):
    if not isinstance(motivation, str):
        return []

    cleaned_motivations = []
    for key, values in motywacje.items():
        for value in values:
            if value in motivation:
                cleaned_motivations.append(key)

    return ", ".join(cleaned_motivations)


df["motivation_cleaned"] = df["motivation"].apply(categorize_motivation)

In [ ]:
motivations_df = df["motivation_cleaned"].str.get_dummies(sep=", ")
motivations_df = motivations_df.drop(columns=["[]"])
motivations_df.head()

In [ ]:
df = pd.concat([df, motivations_df], axis=1)
df.head()

In [ ]:
df.columns

In [ ]:
cleaned_df = df[[
    'age_cleaned',
    'edu_level_cleaned',
    'fav_animals_cleaned',
    'fav_place_cleaned',
    'gender_cleaned',
    'hobby_art',
    'hobby_books',
    'hobby_movies',
    'hobby_other',
    'hobby_sport',
    'hobby_video_games',
    'industry_cleaned',
    'learning_pref_books',
    'learning_pref_chatgpt',
    'learning_pref_offline_courses',
    'learning_pref_online_courses',
    'learning_pref_personal_projects',
    'learning_pref_teaching',
    'learning_pref_teamwork',
    'learning_pref_workshops',
    'learning_preferences',
    'motivation_career',
    'motivation_challenges',
    'motivation_creativity_and_innovation',
    'motivation_money_and_job',
    'motivation_personal_growth',
    'motivation_remote',
    'sweet_or_salty_cleaned',
    'years_of_experience_cleaned',
]].copy()
cleaned_df = cleaned_df.rename(columns={
    'age_cleaned': 'age',
    'edu_level_cleaned': 'edu_level',
    'fav_animals_cleaned': 'fav_animals',
    'fav_place_cleaned': 'fav_place',
    'gender_cleaned': 'gender',
    'industry_cleaned': 'industry',
    'sweet_or_salty_cleaned': 'sweet_or_salty',
    'years_of_experience_cleaned': 'years_of_experience',
})
cleaned_df.head()

In [ ]:
cleaned_df.to_csv('welcome_survey_cleaned_2024_07_27_20_43.csv', index=False, sep=";")